In [29]:
import os
os.getpid()

32017

In [31]:
import random
list2 = []
file_data = open('amazon_review_less_than_300_unbalanced.csv')
for row in file_data:
    list2.append(row)## Data pre-processing module
    
list3 = []
for i in range(len(list2)):
    list3.append((list2[i][-2],list2[i][:-3]))
    
random.seed(10)
random.shuffle(list3)

lenth = len(list3)
train_list = list3[0:int(lenth*0.8)]
test_list = list3[int(lenth*0.8):]




In [32]:

import torch
from torch.utils.data.sampler import BatchSampler, Sampler, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertModel, AdamW, BertConfig
from tqdm import tqdm

class Corpora():
    """
        A class that holds the texts and class labels.
    """
    
    # Modification: l represent train_list or test_list
    
    def __init__(self, l):
        """
            file_path: on MAGIC, it is either
                        data-badassnlp/cola_public/raw/in_domain_train.tsv or
                        data-badassnlp/cola_public/raw/in_domain_dev.tsv
                    on Google Colab, change them to the corresponding paths.
        """

        self.sentences = [s[1] for s in tqdm(l)]
        self.labels = [int(s[0]) for s in tqdm(l)]
        
        
class TextClassificationDataSet(Dataset):
    """
        Define a dataset consisting of pairs of (sequence_of_word_indices, class_label).
        class_label is either 0 or 1.
    """
    def __init__(self, corpora, tokenizer: BertTokenizer):

        self.corpora = corpora
        self.tokenizer = tokenizer

        d =  tokenizer.batch_encode_plus(corpora.sentences)
        
        padded_encoded_inputs = tokenizer.pad(d)
        
        self.input_ids = torch.tensor(padded_encoded_inputs.input_ids)
        self.attention_mask = torch.tensor(padded_encoded_inputs.attention_mask)
        self.labels = torch.tensor(corpora.labels)
        
    def __len__(self):
        return len(self.corpora.sentences)

    def __getitem__(self, idx):
        """
            Return the idx-th of the rows the self.input_ids, self.attention_mask, self.labels in this order.
            Don't do BERT tokenization here as that will be slow.
        """
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [33]:
print(train_list[3000])

('2', 'this product made my soon break out worse:( fast delivery though.. just not right for my skin. good luck with yours:)')


In [34]:
from torch import nn

class BERTClassifier(nn.Module):
    
    def __init__(self, BERT_model, hidden_layer_size, num_classes):
        
        super(BERTClassifier, self).__init__()

        # loaded pretrained model
        self.bert = BERT_model
        
        # simple neural network that convert embedding of the first token to a class
        self.classifier = nn.Sequential(
            nn.Linear(BERT_hidden_size, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        """
        The following two arguments are tensors from a mini-batch of the input_ids
            and attention_mask returned by the BERT tokenizer.
            
            input_ids: a tensor of shape [batch_size, max_length]
            attention_mask: a tensor of shape [batch_size, max_length]
            
            return: the logits of the sentences in the batch tensor of shape [batch_size, 1, 2]
        """
        # see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        # and https://huggingface.co/docs/transformers/main_classes/output
        # for the return value of the forward function of BERT

        ### Your codes go here (30 points) ###
        z = self.bert.forward(input_ids, attention_mask)
        return torch.softmax(self.classifier(z[1]).unsqueeze(1), dim=-1) 
        
        return logits

In [35]:
# make sure you check the availability of GPU when you set the device ID.
device = torch.device(3)

BATCH_SIZE = 8
num_classes = 5
classifier_hidden_size = 128

## if using MAGIC
BERT_PATH = '/data/badassnlp/bert-base-uncased/'
## if using Google Colab, you need to load the bert model after it downloads the model files.
## 

BERT_hidden_size = 768

N_EPOCHS = 5
CLIP = 1

In [36]:
## 'uncased' means all words are lowered-cased before tokenization
## 'base' means the smaller version of BERT (12 layers, 16 heads)
## un-comment one of the following two options.

# if using MAGIC, load from local BERT folder
tokenizer = BertTokenizer.from_pretrained(BERT_PATH, local_files_only=True)
BERT_model = BertModel.from_pretrained(BERT_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False
).to(device)

## if using Colab, load from automatically downloaded files. Downloading can take half a minute
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# BERT_model = BertModel.from_pretrained("bert-base-cased", num_labels = 2, output_attentions = False, output_hidden_states = False).to(device)


## if using MAGIC
print("creating training corpora ... ")
training_corpora = Corpora(train_list)
print("creating training dataset ... ")
training_dataset = TextClassificationDataSet(training_corpora, tokenizer)
print("creating training iterator ... ")
training_iterator = DataLoader(training_dataset, sampler = RandomSampler(training_dataset), batch_size=BATCH_SIZE)

## if using MAGIC
print("creating test corpora ... ")
dev_corpora = Corpora(test_list)
print("creating test dataset ... ")
dev_dataset = TextClassificationDataSet(dev_corpora, tokenizer)
print("creating test iterator ... ")
dev_iterator = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size=BATCH_SIZE)

classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)

Some weights of the model checkpoint at /data/badassnlp/bert-base-uncased/ were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


creating training corpora ... 


100%|██████████| 400000/400000 [00:00<00:00, 1801381.00it/s]


creating training dataset ... 
creating training iterator ... 
creating test corpora ... 


100%|██████████| 100000/100000 [00:00<00:00, 2019706.36it/s]


creating test dataset ... 
creating test iterator ... 


In [39]:
list(training_iterator)[0][0].shape

torch.Size([8, 169])

In [40]:
classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)
classifier.load_state_dict(torch.load("results/best_model_BERT.pt"))



<All keys matched successfully>

In [41]:
from torch import optim
criterion = nn.CrossEntropyLoss(reduction = 'sum')

from tqdm import tqdm

In [45]:

confusion_matrix = []

def evaluate(model, iterator, criterion):

    model.eval()
    confusion_matrix.append(torch.zeros(5,5))
    
    for i, batch in tqdm(enumerate(iterator)):
        
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]-1).to(device)
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) 
        

        for i in range(len(y)):
            row = y[i]
            col = torch.argmax((logits.squeeze(1))[i])
            confusion_matrix[0][row][col] += 1

            
 

In [46]:
print(list(dev_iterator)[0][0].shape)
print(list(dev_iterator)[0][1].shape)

torch.Size([8, 162])
torch.Size([8, 162])


In [48]:
evaluate(classifier, dev_iterator, criterion)   

12500it [06:27, 32.25it/s]


In [49]:
distance = 0
diagnal = 0

for i in range(5):
    for j in range(5):
        distance+= confusion_matrix[0][i][j]*abs(i-j)
        if i == j:
            diagnal += confusion_matrix[0][i][j]
        
print("dist:" ,distance)
print("diag: ",diagnal)

dist: tensor(67306.)
diag:  tensor(110908.)


In [50]:
confusion_matrix

[tensor([[ 8724.,  1532.,   483.,   229.,   321.],
         [ 2055.,  3440.,  1275.,   328.,   240.],
         [  688.,  1629.,  6717.,  2356.,   689.],
         [  237.,   557.,  2265., 14614.,  7963.],
         [  745.,  1584.,  1947., 27713., 77413.]]),
 tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]])]